In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Set up Spotify API client
client_credentials_manager = SpotifyClientCredentials(client_id='30c9485c1d6b4f5e9a4a721df2323f88', client_secret='a7bc3d89a71a4c96ad4c67c81e6e5d35')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# Get user input for seed song
seed_song_name = input("Enter the name of the seed song: ")

# Search for seed song
results = sp.search(q=seed_song_name, limit=1)
seed_song_uri = results['tracks']['items'][0]['uri']

# Get audio features for seed song
seed_features = sp.audio_features(seed_song_uri)[0]

# Search for similar songs
similar_songs = sp.recommendations(seed_tracks=[seed_song_uri], limit=20)['tracks']

Enter the name of the seed song: inch zora


In [4]:
def calculate_similarity(features_1, features_2):
    # Extract relevant features
    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
                    'instrumentalness', 'liveness', 'valence', 'tempo']
    features_1 = [features_1[feature] for feature in feature_list]
    features_2 = [features_2[feature] for feature in feature_list]
    # Normalize features
    features_1 = np.array(features_1).reshape(1, -1)
    features_2 = np.array(features_2).reshape(1, -1)
    features_1 = (features_1 - np.mean(features_1)) / np.std(features_1)
    features_2 = (features_2 - np.mean(features_2)) / np.std(features_2)
    # Calculate cosine similarity
    return cosine_similarity(features_1, features_2)[0][0]


In [5]:
# Calculate similarity scores for each similar song
similarity_scores = []
for song in similar_songs:
    # Get audio features for song
    song_features = sp.audio_features(song['uri'])[0]
    # Calculate similarity score
    similarity_score = calculate_similarity(seed_features, song_features)
    similarity_scores.append((song['name'], song['external_urls']['spotify'], similarity_score))

In [6]:
# Sort similar songs by similarity score
sorted_songs = sorted(similarity_scores, key=lambda x: x[2], reverse=True)

In [7]:
# Display top 5 similar songs
print("Top 5 similar songs to", seed_song_name, ":\n")
for i in range(5):
    song_name, song_url, similarity_score = sorted_songs[i]
    print(f"{i+1}. {song_name} ({song_url}) - similarity score: {similarity_score:.2f}")

Top 5 similar songs to inch zora :

1. Black Matte (https://open.spotify.com/track/3HHOY3hOZRJf1ORqNSDS8B) - similarity score: 1.00
2. Kamli (https://open.spotify.com/track/15fmoePTSxMrZCskvyXSac) - similarity score: 1.00
3. Le Chakk Main Aa Gya (https://open.spotify.com/track/4EdPSOsTEUDplToG2AIfyl) - similarity score: 1.00
4. Kali Denali (https://open.spotify.com/track/3ZJBsQMWvesJ6w8UqalKVL) - similarity score: 1.00
5. Do Vaari Jatt (https://open.spotify.com/track/68BrFM98WyShOImDZ58sah) - similarity score: 1.00
